# Pro-tips- the challenge here is that with the netcdf in its origional configuration it was not possible to read all the data in to downsample it. So what we did was to read in each file, swap the time and obs dimensions and then only keep the 
1. Create a directory in you data files for downsampled .ncdfs to live

In [3]:
import xarray as xr
import pandas as pd
import dask.dataframe as dd
import re
import requests
import pickle as pk
import os
import gc
#!pip install netcdf4 == 1.5.0

In [4]:
# make the output directory
new_dir = '/home/jovyan/data/botpt/minute_mean_dataD/'
if not os.path.isdir(new_dir):
    try:
        os.makedirs(new_dir)
    except OSError:
        if os.path.exists(new_dir):
            pass
        else:
            raise

In [5]:
# Sensor E: url = 'https://opendap.oceanobservatories.org/thredds/catalog/ooi/dax.soule@qc.cuny.edu/20181104T104012-RS03ECAL-MJ03E-06-BOTPTA302-streamed-botpt_nano_sample/catalog.html'
# Sensor B url = 'https://opendap.oceanobservatories.org/thredds/catalog/ooi/dax.soule@qc.cuny.edu/20181020T213838-RS03ASHS-MJ03B-09-BOTPTA304-streamed-botpt_nano_sample/catalog.html'
# Sensor C url = 'https://opendap.oceanobservatories.org/thredds/catalog/ooi/dax.soule@qc.cuny.edu/20181104T041943-RS03CCAL-MJ03F-05-BOTPTA301-streamed-botpt_nano_sample/catalog.html'
url = 'https://opendap.oceanobservatories.org/thredds/catalog/ooi/dax.soule@qc.cuny.edu/20191015T040958136Z-RS03INT2-MJ03D-06-BOTPTA303-streamed-botpt_nano_sample/catalog.html'

In [6]:
tds_url = 'https://opendap.oceanobservatories.org/thredds/dodsC'
datasets = requests.get(url).text
urls = re.findall(r'href=[\'"]?([^\'" >]+)', datasets)
x = re.findall(r'(ooi/.*?.nc)', datasets)
for i in x:
    if i.endswith('.nc') == False:
        x.remove(i)
for i in x:
    try:
        float(i[-4])
    except:
        x.remove(i)
datasets = [os.path.join(tds_url, i + '#fillmismatch') for i in x]
#datasets

## origional loop

In [5]:
# num = 0
# for i in datasets:
#     ds = xr.open_dataset(i)
#     ds = ds.swap_dims({'obs': 'time'})

#     pressure_min = pd.DataFrame()
#     pressure_min['bottom_pressure'] = ds['bottom_pressure'].to_pandas().resample('T').mean()
#     del pressure_min.index.name

#     pressure_min = pressure_min.dropna()

#     out = 'minute_mean_data/' + i.split('/')[-1][:-3] + '_resampled' + '.pd'
#     num = num +1

#     with open(out, 'wb') as fh:
#         pk.dump(pressure_min,fh)

#     gc.collect()

# pd routine

In [ ]:
%%time
for i in datasets:
    ds = xr.open_dataset(i)
    ds = ds.swap_dims({'obs': 'time'})

    pressure_min = pd.DataFrame()
    pressure_min['bottom_pressure'] = ds['bottom_pressure'].to_pandas().resample('T').mean()
    del pressure_min.index.name

    pressure_min = pressure_min.dropna()

    out = '/home/jovyan/data/botpt/minute_mean_dataD/' + i.split('/')[-1][:-3] + '_resampled' + '.nc'
    ds = xr.Dataset.from_dataframe(pressure_min)
    ds.to_netcdf(out)

# Test Loop PD

In [ ]:
# %%time
# i=datasets[1]
# ds = xr.open_dataset(i)
# ds = ds.swap_dims({'obs': 'time'})

# pressure_min = pd.DataFrame()
# pressure_min['bottom_pressure'] = ds['bottom_pressure'].to_pandas().resample('T').mean()
# del pressure_min.index.name

# pressure_min = pressure_min.dropna()
# out = 'minute_mean_data/test.nc'

# ds = xr.Dataset.from_dataframe(pressure_min)
# ds.to_netcdf(out)

## xr routine 

In [ ]:
# %%time
# for i in datasets:
#     ds = xr.open_mfdataset([i], parallel =True, combine='by_coords')
#     ds = ds.swap_dims({'obs': 'time'})
#     ds= ds['bottom_pressure'].to_dataset()
#     ds=ds.resample(time='T').mean()
#     out = 'minute_mean_data/' + i.split('/')[-1][:-3] + '_resampled_test' + '.nc'
#     ds.to_netcdf(out)

distributed.client - WARNING - Couldn't gather 2 keys, rescheduling {'open_dataset-c373e84a-6ea6-42e2-b50d-25b833c29dcf': ('tcp://10.0.128.243:36847',), 'getattr-889ccf3e-e4de-4116-8580-aa80ca6f3e29': ('tcp://10.0.128.243:36847',)}
distributed.client - WARNING - Couldn't gather 2 keys, rescheduling {'open_dataset-c373e84a-6ea6-42e2-b50d-25b833c29dcf': ('tcp://10.0.128.101:46587',), 'getattr-889ccf3e-e4de-4116-8580-aa80ca6f3e29': ('tcp://10.0.128.101:46587',)}
distributed.client - WARNING - Couldn't gather 2 keys, rescheduling {'open_dataset-8a478b4d-fb45-4bfc-a163-9f16bb513072': ('tcp://10.0.129.20:37071',), 'getattr-84a8c166-e131-43a7-9a55-e79cf1ded9b4': ('tcp://10.0.129.20:37071',)}


# Least Efficienct

In [ ]:
# %%time
# i= datasets[1]
# ds = xr.open_mfdataset([i], parallel=True, drop_variables= ['deployment', 'id', 'bottom_pressure_qc_executed', 'date_time_string'
#                                                   , 'driver_timestamp', 'ingestion_timestamp', 'internal_timestamp'
#                                                   ,'port_timestamp', 'preferred_timestamp', 'press_trans_temp'
#                                                   ,'provenance','sensor_id', 'time_sync_flag', 'bottom_pressure_qc_results']
#                             )
# ds=ds.swap_dims({'obs': 'time'}).resample(time='T').mean()
# out = 'minute_mean_data/' + i.split('/')[-1][:-3] + '_resampled' + '.nc'
# ds.to_netcdf(out)

In [2]:
!pwd

/home/jovyan/repos/botpt/manuscript


In [1]:
!rm minute_mean_data/*

rm: cannot remove 'minute_mean_data/*': No such file or directory


In [ ]:
ds = xr.open_mfdataset('minute_mean_data/*_resampled.nc', parallel=True,combine='by_coords')
#ds = ds.swap_dims({'obs': 'time'}.resample(time='10T').mean()
ds

In [ ]:
df = ds.to_dask_dataframe()

In [ ]:
df.head()